In [2]:
import pandas as pd
import numpy as np
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from itertools import chain
import re

In [3]:
# Scraping story text

my_url = 'https://www.projekt-gutenberg.org/fallada/hoppelpo/chap001.html'

uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

page_soup = soup(page_html, 'html.parser')

In [4]:
container = page_soup.find_all('p')
container = container[0]

In [5]:
title = container.find_all('h3')
title = str(title[0])

In [6]:
story = container.find_all('p')
paragraphs = []
for i in story:
    paragraphs.append(str(i))

In [7]:
for i in range(len(paragraphs)):
    paragraphs[i] = paragraphs[i].replace('<p>', '')
    paragraphs[i] = paragraphs[i].replace('</p>', '') 

In [8]:
words = []
try:
    for i in range(len(paragraphs)):
        for j in range(len(paragraphs[i])):
            words.append(paragraphs[j].split(' ')) 
except:
    pass

In [9]:
# Flatten nested lists in words

words = list(chain.from_iterable(words))

In [10]:
for i in range(len(words)):
        words[i] = words[i].strip()
        words[i] = words[i].replace('\xa0...', '')
        words[i] = words[i].replace('<a', '')
        words[i] = words[i].replace('</a>', '')
        words[i] = words[i].replace('\n', '')
        words[i] = words[i].replace('-', '')
        words[i] = words[i].replace('–', '')
        words[i] = words[i].replace('.', '')
        words[i] = words[i].replace(':', '')
        words[i] = words[i].replace(',', '')
        words[i] = words[i].replace('>', '')
        words[i] = words[i].replace('<', '')
        words[i] = words[i].replace('›', '')
        words[i] = words[i].replace('‹', '')
        words[i] = words[i].replace('?', '')
        words[i] = words[i].replace('!', '')
        words[i] = words[i].replace('«', '')
        words[i] = words[i].replace('»', '')
        words[i] = words[i].replace('(', '')
        words[i] = words[i].replace(')', '')
        words[i] = words[i].replace(' ', '')
        words[i] = re.sub(str('id='+'"'+'page\d'+'"'), '', words[i])
        words[i] = re.sub(str('name='+'"'+'page\d'+'"'), '', words[i])

In [61]:
# Remove empty strings in words

while("" in words) :
    words.remove("")

In [163]:
df = pd.DataFrame(words)

In [164]:
df.reset_index(inplace = True)

In [165]:
#df.columns = ['rank', 'text']
df.columns = ['rank', 'text']
df['text2'] = ''

In [166]:
# Cleaning last HTML code

part1 = df[df.text == 'title="Dieter7/lac"']
part2 = df[df.text == 'title="suru/Dieter7"']
part3 = df[df.text == 'title="lac/Dieter7"']
eraser = pd.concat([part1, part2, part3])

erank = []
for i in range(len(eraser)):
    erank.append(int(eraser.index[i]))

for i in erank:
    df = df.drop([i])

In [167]:
df.reset_index(drop = True, inplace = True)

In [168]:
df = df.drop('rank', axis = 1)

In [169]:
df.reset_index(inplace = True)

In [170]:
df.columns = ['rank', 'text', 'text2']

In [172]:
# Fill 'text2' with text from previous row

for i in range(len(df)-1):
    if i < len(df):
        try:
            df['text2'][i] = df.text[i+1] 
        except:
            pass

<ipython-input-172-d31fb755fdcd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text2'][i] = df.text[i+1]


In [173]:
df

,rank,text,text2
0,0,Es,war
1,1,war,einmal
2,2,einmal,ein
3,3,ein,kleiner
4,4,kleiner,Junge
...,...,...,...
1294,1294,die,Welt
1295,1295,Welt,wieder
1296,1296,wieder,in
1297,1297,in,Ordnung
